Import necessary libraries
You will need ***numpy*** and ***matplotlib***

In [1]:
import numpy as np
from matplotlib import pyplot as plt

MPA class contains all necessary functions for processing with one-dimensional tensor networks.
MPA is a direct high dimensional generalization of MPS (MPS has one physical index whereas MPA has multiple physical indices.)
We can organize our class as follows:
- the object of the class contains a list of local tensors (in the case of MPS one corresponds to $\left\{A^{(n)}_{ikj}\right\}_{n=1}^{N}$),, number of states, the shape of "physical" space, list of bond dims.
- Objects of MPA class must have the same shape of all site physical indices, bond dims can be arbitrary
- MPA class includes the following list of methods and functions: reshape, transpose, complex conjugate, einsum (local convolutions), evaluate, truncation, ability to create random mpa, etc.
 As a part of the task, you have to fill gaps in the code below (№0): Import necessary libraries. 
 We recommend to take a time and examine in details all methods in the follwoing class.

In [13]:
class MPA:
    
    
    '''the init function initializes an object of the class. As an input, the function takes
    list of local tensors of shape (left, n1, n2, ..., nk, right), where left is the left bond dim.,
    right is the right bond dim. and n1, n2, ..., nk is the shape of physical space.'''
    def __init__(self, list_of_tensors):
        
        #the list of local tensors in the body of an object
        self.list_of_tensors = list_of_tensors
        
        #the number of sites in the body of an object
        self.num_sites = len(list_of_tensors)
        
        #here we check that physical shapes of each local tensor are the same
        for i in range(1, self.num_sites):
            if list_of_tensors[i-1].shape[1:-1] != list_of_tensors[i].shape[1:-1]:
                raise ValueError("INCORRECT ONE SITE SHAPES")
        
        #here we check that bond dims of all local tensors are "stackable"
        for i in range(1, self.num_sites):
            if list_of_tensors[i-1].shape[-1] != list_of_tensors[i].shape[0]:
                raise ValueError("INCORRECT BOND DIM.")
        
        #the site shape (shape of physical space) in the body of an object
        self.site_shape = list_of_tensors[0].shape[1:-1]
        
        #the list of bond dims in the body of aa object
        self.bond_dims = [list_of_tensors[0].shape[0]] + [tensor.shape[-1] for tensor in list_of_tensors]
        
    '''the method reshapes the physical part of a local tensor, as an input, 
                  the method takes new shape (tuple) The method returns MPA object with a new shape.'''
    def reshape(self, shape):
        #the following list will be filled by reshaped local tensors
        list_of_tensors = []
        
        #the loop over all local tensors
        for i, tensor in enumerate(self.list_of_tensors):
            
            list_of_tensors.append(np.reshape(tensor, shape))
            
            #here you have to fill list_of_tensors by reshaped local tensors
            #(hint: use np.reshape and atribute .shape)
            # ENTER YOUR CODE ##
            
        return MPA(list_of_tensors)
    
    '''the method transposes the phycical part of a local tensor, as an input the method takes new order of physical
    indices. The method returns MPA object with a new shape.'''
    def transpose(self, tuple_of_indeces):
        #the following list will be filled by transposed local tensors
        list_of_tensors = []
        indeces = tuple((i+1 for i in tuple_of_indeces))
        
        #the loop over all local tensors
        for i, tensor in enumerate(self.list_of_tensors):
            tensor = np.transpose(tensor, indeces)
            
            list_of_tensors.append(tensor[1:-1])
            
            # avoid any transpositions of bond indices, operate only with physical ones! You can encode 
            # your transpostion of phys. indeces wich must be taken as tuple
            # ENTER YOUR CODE
        return MPA(list_of_tensors)

    '''the method perform complex conjugation of all local tensors.''' 
    def conj(self):
        #the following  list must be filled by conj. local tensors 
        list_of_tensors = []
        #use statndart method of numpy lib  -.conj # 
        for i, tensor in enumerate(self.list_of_tensors):
            list_of_tensors.append(tensor.conj())
        return MPA(list_of_tensors)
   
    '''the method performs physical indeces arbitrary convolution of two adjustent local tensors. 
    It is generalization on numpy's np.einsum''' 
    @staticmethod   
    def einsum(string, mpa1, mpa2):
        
        #preparing einsum string for local tensors
        #here we give you hint how deal with bond indices
        inp, out = string.split('->')
        inp_a, inp_b = inp.split(',')
        corr_inp_a = 'w' + inp_a + 'x'
        corr_inp_b = 'y' + inp_b + 'z'
        corr_out = 'wy' + out + 'xz'
        corr_string = corr_inp_a + ',' + corr_inp_b + '->' + corr_out
        
        #Here we ask - are the lengths of mps arrays equal?
        assert len(mpa1.list_of_tensors) == len(mpa2.list_of_tensors), 'mp arrays have different lenght'
        
        #the following list must be filled by convoluted local tensors 
        new_list_of_tensors = []
        #loop over all local tensors #
        for i in range(len(mpa1.list_of_tensors)):
            #use string ( corr_string) below as argument of numpy's np.einsum 
            # ENTER YOUR CODE ##
            #Here we reshape bond dims to give a correct form of new local tensors 
            tensor = np.einsum(corr_string, [mpa1.list_of_tensors[i], mpa1.list_of_tensors[i]])
            shape = tensor.shape
            tensor = tensor.reshape((-1,) + shape[2:-2] + (shape[-2]*shape[-1],))
            new_list_of_tensors.append(tensor)
            
        
        return MPA(new_list_of_tensors)
        
    
   
    '''This method allows one to create a random MPA '''
    @staticmethod
    
    def random_mpa(num_sites, site_shape, bond_dim):
        
        left_shape = (1,) + site_shape + (bond_dim,)
        mid_shape =  (bond_dim,) + site_shape + (bond_dim,)
        right_shape = (bond_dim,) + site_shape + (1,)
        left_tensor = np.random.randn(*left_shape) + np.random.randn(*left_shape) * 1j
        mid_tensor = np.random.randn(*mid_shape) + np.random.randn(*mid_shape) * 1j
        right_tensor = np.random.randn(*right_shape) + np.random.randn(*right_shape) * 1j
        list_of_tensors = [left_tensor] + (num_sites - 2) * [mid_tensor] + [right_tensor]
        return MPA(list_of_tensors)
    
    '''This method allows one to return value of local tensor with respect to the given index'''
    def value_mps(self, indeces):
        assert len(self.list_of_tensors[0].shape[1:-1]) == 1, "GIVEN MPA IS NOT MPS"
        in_tensor = self.list_of_tensors[0][:, indeces[:, 0], :]
        for i in range(1, len(self.list_of_tensors)):
            update_tensor = self.list_of_tensors[i][:, indeces[:, i], :]
            in_tensor = np.einsum('ijk,kjl->ijl', in_tensor, update_tensor)
        return in_tensor.flatten()
    '''This method allows one to calculate a norm of MPS state. Note, this method  is inplace method!!!'''
    def norm_psi(self):
        assert len(self.list_of_tensors[0].shape[1:-1]) == 1, "GIVEN MPA IS NOT MPS"
        local_tensor = self.list_of_tensors[0]
        in_tensor = np.einsum('ijk,ljm->ilkm', local_tensor, local_tensor.conj())
        norm = np.linalg.norm(in_tensor)
        in_tensor = in_tensor / norm
        self.list_of_tensors[0] = local_tensor / np.sqrt(norm)
        for i in range(1, len(self.list_of_tensors)):
            update_tensor = np.einsum('ijk,ljm->ilkm', self.list_of_tensors[i], self.list_of_tensors[i].conj())
            in_tensor = np.einsum('ijkm,kmln->ijln', in_tensor, update_tensor)
            norm = np.linalg.norm(in_tensor)
            self.list_of_tensors[i] = self.list_of_tensors[i] / np.sqrt(norm)
            in_tensor = in_tensor / norm

    '''This method allows one to convolute MPS'''
    def evaluate(self):
        assert len(self.site_shape) == 0, "MPA IS NOT FULLY CONVOLUTED"
        local_tensor = self.list_of_tensors[0]
        local_tensor = local_tensor.reshape((local_tensor.shape[0],) + (local_tensor.shape[-1],))
        for i in range(1, len(self.list_of_tensors)):
            update_tensor = self.list_of_tensors[i]
            update_tensor = update_tensor.reshape((update_tensor.shape[0],) + (update_tensor.shape[-1],))
            local_tensor = local_tensor.dot(update_tensor)
        return local_tensor.flatten()
    

### The part №1
You have to experiment and get comfortable with basic operations over MPA/MPS

In [ ]:
#Test part:
np.random.seed(11)
#Create random mps for spin chain of size N = 32, bond dims = 5
a = 
#Culculate conjugated state 
# ENTER YOUR CODE
#Perform convolution of original mps with conjugated one over all physical indeces
conv = # ENTER YOUR CODE
print(np.real(conv.list_of_tensors[0][0, 0])) #THIS NUMBER IS AN ANSWER FOR THE TEST!

In [ ]:
#Create random mps for spin chain of size N = 32, bond dims = 5 
mps1 = # ENTER YOUR CODE
#Perform Normalization procedure over your mps1
# ENTER YOUR CODE
#Culculate conjugated state 
# ENTER YOUR CODE
#Calculate normalization coefficient manualy: norm = <psi|psi> (after normalization of state norm must be equal to 1)
norm = # ENTER YOUR CODE
# ENTER YOUR CODE

Final Task: Consider GHZ state. Using ypur class represent GHZ state in MPS from.
Eneter you reslts for N=100 particles:
1. num_sites = 
2. list of dims = 
3. normalization constant = 

In [ ]:
#GHZ in MPS form

# ENTER YOUR CODE

In [22]:
a = np.random.random((2, 3, 3, 3, 5))

In [23]:
a

array([[[[[0.67939823, 0.55132564, 0.18474731, 0.41438619, 0.40460274],
          [0.06134884, 0.02303   , 0.76910049, 0.0031951 , 0.51936809],
          [0.12899578, 0.15945217, 0.91365345, 0.83710024, 0.57161891]],

         [[0.86470308, 0.41951239, 0.7209467 , 0.68869181, 0.79671008],
          [0.23658186, 0.4384978 , 0.27871575, 0.53019645, 0.5535736 ],
          [0.94808938, 0.74692308, 0.64649718, 0.98536712, 0.80461046]],

         [[0.90634039, 0.35383307, 0.28359699, 0.95557046, 0.5042442 ],
          [0.11830508, 0.77593824, 0.82328656, 0.7049851 , 0.02271733],
          [0.38932741, 0.17325757, 0.770582  , 0.96573829, 0.85120376]]],


        [[[0.50973694, 0.65634052, 0.08573755, 0.03123045, 0.73562089],
          [0.59707135, 0.98266737, 0.29495888, 0.58025502, 0.07837281],
          [0.03624149, 0.74070619, 0.71423542, 0.0861823 , 0.06577721]],

         [[0.39657304, 0.33912158, 0.44434833, 0.76392463, 0.24825352],
          [0.44693116, 0.12633192, 0.18452637, 0.45575

In [30]:
idx = [1,2]

In [32]:
b = np.transpose(a, axes = idx)

ValueError: axes don't match array